# Interacting with Coq

The `Pytanque` class implements a lightweight client that can communicate with the Rocq prover via the `pet-server`.

In [1]:
from pytanque import Pytanque

First you need to start the server.
Instructions can be found in the coq-lsp repo (https://github.com/ejgallego/coq-lsp/tree/main/petanque)

Then, you can create a client and connect it to the same address and port.

In [2]:
pet = Pytanque("127.0.0.1", 8765)
pet.connect()

Petanque relies on regular coq files.
To prove an existing theorem, just indicate the location of the `.v` file and the name of the theorem.

In [3]:
file_name = "scratch.v"
thm_name = "test"
thm_body = "forall n : nat, 2 * n = n + n."

with open(f"./{file_name}", "w") as file:
    print(f"Lemma {thm_name} : {thm_body}", file=file)


state = pet.start(file=file_name, thm=thm_name)

Internally the state is a simple `id` (and an optional `hash`).
To retrieve the state of the prover, you can use the `goals` method.

In [4]:
print(f"Internal pytanque state: {state}")

goals = pet.goals(state)  # Retrieve actual state
print(f"Prover goals: {goals}\n")
for g in goals:
    print(g.pp)  # Pretty print the current goals

Internal pytanque state: State(st=9, proof_finished=False, hash=1053139973)
Prover goals: [Goal(info={'evar': ['Ser_Evar', 4], 'name': None}, hyps=[GoalHyp(names=['n'], ty='nat', def_=None)], ty='2 * n = n + n')]

n  : nat
|-2 * n = n + n


To execute a tactic, use the `run_tac` method.

In [5]:
state = pet.run_tac(state, "induction n. simpl.")
goals = pet.goals(state)
for g in goals:
    print(g.pp)  # Pretty print the current goals
    print("\n")


|-2 * 0 = 0 + 0


n  : nat
IHn  : 2 * n = n + n
|-2 * S n = S n + S n




You can also use the `verbose` option of `run_tac` to retrieve and print the current goal after executing a tactic.

In [6]:
state = pet.run_tac(state, "auto.", verbose=True)


Goal 0:
n  : nat
IHn  : 2 * n = n + n
|-2 * S n = S n + S n



If the proof is finished (no more goal) the field `proof_finished` becomes `True`.

In [7]:
state = pet.run_tac(state, "auto.")
state = pet.run_tac(state, "simpl; auto.")
state

State(st=13, proof_finished=True, hash=24173511)

# Proof search

Adapted from the NeurIPS 2024 tutorial: https://github.com/yangky11/ml4tp-tutorial/blob/main/main.ipynb 

Let us start with a dummy tactic generator (random sampling in a list of tactics) with random scores for each tactics in $[0, 1]$.

In [20]:
import random
import pytanque as pt
from typing import List, Tuple, Optional

Tactic = str
Proof = List[Tactic]
Score = float


def generate_tactics(
    goal: pt.State, num_candidates: int
) -> Tuple[List[str], List[float]]:
    tactics = [
        "intros.",
        "auto.",
        "simpl.",
        "induction n.",
        "easy.",
        "trivial.",
        "rewrite IHn.",
        # "destruct IHn.",
        "lia.",
    ]
    return random.choices(tactics, k=num_candidates), [
        random.uniform(0, 1) for _ in range(num_candidates)
    ]

We can now implement depth-first search to search for proofs.

In [21]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)


def depth_first_search(state: pt.State, max_depth: int, num_candidates: int):
    run_ops = pt.Opts(memo=False, hash=False)  # deactivate memoization and hashing

    def search(state: pt.State, depth: int) -> Optional[Proof]:
        if depth >= max_depth:
            return None

        tactics, _ = generate_tactics(state, num_candidates)

        # Run the tactics.
        for tac in tactics:
            try:
                next_state = pet.run_tac(state, tac, run_ops)

                if next_state.proof_finished:
                    return [tac]

                subproof = search(next_state, depth + 1)
                if subproof:
                    return [tac] + subproof

            except pt.PetanqueError as err:
                logging.warning(f"Invalid Tactic: {tac} {err}")

        return None

    return search(state, depth=0)

Let us try on a simple example.

In [22]:
pre_commands = "From Coq Require Import Lia."
state = pet.start(file_name, thm_name, pre_commands)
proof = depth_first_search(state, max_depth=10, num_candidates=16)

if proof:
    print(f"Proof: {" ".join(proof)}")
else:
    print("Not proof found...")

Proof: trivial. induction n. intros. trivial. auto. auto. destruct IHn. trivial. simpl. auto.


Here is an implementation of best-first search adapted from the 2023 Neural theorem proving tutorial https://github.com/wellecks/ntptutorial

In [25]:
import heapq
from tqdm import trange


def best_first_search(
    init_state: pt.State, max_iters: int, num_samples: int
) -> Optional[Proof]:
    queue = [(0.0, [], init_state)]
    visited = set()
    for _ in trange(max_iters):
        print("Queue:", queue)

        if len(queue) == 0:
            break

        total_score, steps, state = heapq.heappop(queue)
        visited.add(state.hash)

        step_cands, step_scores = generate_tactics(state, num_samples)
        print(step_cands)
        print("VISITED:", visited)

        for step, score in zip(step_cands, step_scores):
            try:
                print(f"try {step}")
                result = pet.run_tac(state, step)
                print(result.hash)

                if result.proof_finished:
                    return steps + [step]

                if result.hash not in visited:
                    print(f"Add result of {step} to queue")
                    # Score is negative log probability summed across steps
                    new_score = total_score - score
                    heapq.heappush(queue, (new_score, steps + [step], result))

            except pt.PetanqueError as err:
                print(err)
                logging.warning(f"Invalid Tactic: {step} {err}")
    return None

In [26]:
init_state = pet.start(file_name, thm_name, pre_commands)
proof = best_first_search(init_state, max_iters=10, num_samples=10)

if proof:
    print(f"Proof: {" ".join(proof)}")
else:
    print("Not proof found...")

 10%|█         | 5/50 [00:00<00:00, 172.79it/s]

Proof: induction n. simpl. trivial. destruct IHn. simpl. auto.
